In [19]:
!pip install --upgrade --quiet google-genai

In [20]:
!pip install gTTS --quiet

In [21]:
!pip install playsound --quiet

In [22]:
from google.colab import userdata
GEMINI_API_KEY : str = userdata.get('GEMINI_API_KEY')
if GEMINI_API_KEY:
  print("API KEY FOUND")
else:
  print("API KEY NOT FOUND")

API KEY FOUND


In [23]:
from google import genai
from google.genai import Client

client : Client = genai.Client(
    api_key = GEMINI_API_KEY,
)

model : str = "gemini-2.0-flash-exp"

In [24]:
video_file_name = "/content/Self Introduction Video.mp4"

In [25]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path = video_file_name)
  while(video_file.state == "PROCESSING"):
    print('Waiting for video to be processed.')
    time.sleep(10)
    video_file = client.files.get(name = video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f"Video Processing Complete : " + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)

Waiting for video to be processed.
Video Processing Complete : https://generativelanguage.googleapis.com/v1beta/files/r9v3nai6gpsk


In [26]:
from google.genai.types import Content, Part
prompt = """For each scene in this video.
            Generate captions that describes the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video."""

video = my_video
response = client.models.generate_content(
    model = model,
    contents = [
        Content(
            role = "user",
            parts = [
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""
                ),
            ]
        ),
        prompt,
    ]
)

print(response.text)

Okay, here are the captions for each scene in the video, in the format you requested:

```json
[
  {
   "timecode": "00:00",
   "caption": "A man is seen smiling at the camera, he is wearing a light-colored collared shirt and a red and white lanyard with a school name. Behind him is a green wall with black-framed bulletin boards and writing in a cursive font. He is laughing and then begins to speak."
  },
 {
    "timecode": "00:00",
    "caption":"The man raises a hand and then places it on his chest. He says, “Hi everyone. My name is Angel D. Caberra Junior. I was born and raised here in the Philippines.”"
   },
   {
    "timecode": "00:07",
    "caption":"He continues with, “I am currently a public school teacher here and I am also a part time college instructor at Eastern Visayas State University.”"
   },
  {
    "timecode": "00:15",
    "caption":"He says, “I have eight years of teaching experience, and still counting.”"
  },
    {
     "timecode": "00:19",
    "caption":"He says, 

In [31]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

tts = gTTS(text = "\"Hello, my name is Zohaib Khan. I'm currently pursuing my Bachelor's in Data Science at Punjab University. My primary focus is on Python programming, and I'm deeply interested in exploring new technologies, particularly in the field of AI. My long-term goal is to become an agentic AI developer. I'm eager to connect with others in the field and learn more about the latest advancements.\""
,lang = 'en')

with open("output.mp3","wb") as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio("output.mp3",autoplay=True))